#### Q1. Getting the embeddings model
##### Q1 answer = -0.42

In [ ]:
import pandas as pd

github_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [68]:
df = df.iloc[:300]

In [69]:
df

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp
...,...,...,...,...,...
295,An alternative way to load the data using the ...,Above users showed how to load the dataset dir...,8d209d6d,What is an alternative way to load the data us...,machine-learning-zoomcamp
296,You can directly download the dataset from Git...,Above users showed how to load the dataset dir...,8d209d6d,How can I directly download the dataset from G...,machine-learning-zoomcamp
297,You can fetch data for homework using the `req...,Above users showed how to load the dataset dir...,8d209d6d,Could you share a method to fetch data for hom...,machine-learning-zoomcamp
298,If the status code is 200 when downloading dat...,Above users showed how to load the dataset dir...,8d209d6d,What should I do if the status code is 200 whe...,machine-learning-zoomcamp


In [49]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

In [50]:
answer_llm = df.iloc[0].answer_llm

In [51]:
answer_llm

'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).'

In [52]:
embeddings = embedding_model.encode(answer_llm)

In [53]:
# Q1 answer = -0.42
embeddings[:5]

array([-0.42244664, -0.22485578, -0.3240585 , -0.2847588 ,  0.00725704],
      dtype=float32)

#### Q2. Computing the dot product
##### Q2 answer = 31.67

In [56]:
import numpy as np

evaluations = []
for index, row in df.iterrows():
    if index%50==0:
        print(f"processing {index}...")
    answer_llm = row.answer_llm
    answer_orig = row.answer_orig
    answer_llm_embed = embedding_model.encode(answer_llm)
    answer_orig_embed = embedding_model.encode(answer_orig)
    
    result = answer_llm_embed.dot(answer_orig_embed)
    evaluations.append(result)
    
print("done")

processing 0...
processing 50...
processing 100...
processing 150...
processing 200...
processing 250...
done


In [57]:
evaluations[:5]

[17.515999, 13.418407, 25.313251, 12.147418, 18.74773]

In [60]:
# Q2 answer = 31.67
series = pd.Series(evaluations)
stats = series.describe()
stats

count    300.000000
mean      27.495996
std        6.384744
min        4.547924
25%       24.307847
50%       28.336860
75%       31.674305
max       39.476021
dtype: float64

#### Q3. Computing the cosine
##### Q3 answer = 0.83

In [63]:
import numpy as np

def get_v_norm(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

evaluations = []
for index, row in df.iterrows():
    if index%50==0:
        print(f"processing {index}...")
    answer_llm = row.answer_llm
    answer_orig = row.answer_orig
    answer_llm_embed = embedding_model.encode(answer_llm)
    answer_orig_embed = embedding_model.encode(answer_orig)
    
    answer_llm_norm = get_v_norm(answer_llm_embed)
    answer_orig_norm = get_v_norm(answer_orig_embed)
    
    result = answer_llm_norm.dot(answer_orig_norm)
    evaluations.append(result)
    
print("done")
evaluations[:5]

processing 0...
processing 50...
processing 100...
processing 150...
processing 200...
processing 250...
done


[0.5067543, 0.388549, 0.7185991, 0.3372664, 0.5217923]

In [64]:
# Q3 answer = 0.83
series = pd.Series(evaluations)
stats = series.describe()
stats

count    300.000000
mean       0.728392
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
dtype: float64

#### Q4. Rouge
##### Q4 answer = 0.45

In [70]:
doc = df.iloc[10]
doc

answer_llm     Yes, all sessions are recorded, so if you miss...
answer_orig    Everything is recorded, so you won’t miss anyt...
document                                                5170565b
question                    Are sessions recorded if I miss one?
course                                 machine-learning-zoomcamp
Name: 10, dtype: object

In [71]:
# Q4 ans=0.45

from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(doc['answer_llm'], doc['answer_orig'])[0]
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

#### Q5. Average rouge score
##### Q5 answer = 0.35

In [73]:
# Q5 ans=0.35
f_scores = [metric['f'] for metric in scores.values()]
average_f_score = sum(f_scores) / len(f_scores)
average_f_score

0.35490034990035496

#### Q6. Average rouge score for all the data points
##### Q6 answer = 0.2

In [76]:
import numpy as np

evaluations = []
for index, row in df.iterrows():
    if index%50==0:
        print(f"processing {index}...")
    answer_llm = row.answer_llm
    answer_orig = row.answer_orig
    scores = rouge_scorer.get_scores(answer_llm, answer_orig)[0]
    result = scores["rouge-2"]["f"]
    
    evaluations.append(result)
    
print("done")
evaluations[:5]

processing 0...
processing 50...
processing 100...
processing 150...
processing 200...
processing 250...
done


[0.028169010918468917,
 0.05555555225694465,
 0.17777777313333343,
 0.047058819111419105,
 0.03389830142092829]

In [77]:
# Q6 ans = 0.2
series = pd.Series(evaluations)
stats = series.describe()
stats

count    300.000000
mean       0.206965
std        0.153550
min        0.000000
25%        0.097809
50%        0.178671
75%        0.286181
max        0.739130
dtype: float64